In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
url = "mysql+mysqlconnector://root@127.0.0.1:3306/training"

engine = create_engine(url)
connection = engine.connect()

In [3]:
query = text("SELECT * FROM business;")

result = connection.execute(query)
df_bus = pd.DataFrame(result.fetchall())
df_bus.head()

,business,year_founded,country_code,category_code
0,Hamoud Boualem,1878,DZA,CAT11
1,Hamoud Boualem,1878,DZA,CAT11\r
2,Communauté Électrique du Bénin,1968,BEN,CAT10\r
3,Botswana Meat Commission,1965,BWA,CAT1\r
4,Air Burkina,1967,BFA,CAT2\r


In [4]:
query = text("SELECT * FROM countries;")

result = connection.execute(query)
df_cont = pd.DataFrame(result.fetchall())
df_cont.head()

,country_code,country,continent
0,AFG,Afghanistan,Asia
1,AGO,Angola,Africa
2,ALB,Albania,Europe
3,AND,Andorra,Europe
4,ARE,United Arab Emirates,Asia


In [5]:
query = text("SELECT * FROM categories;")

result = connection.execute(query)
df_cat = pd.DataFrame(result.fetchall())
df_cat.head()

,category,category_code
0,Agriculture,CAT1
1,Agriculture,CAT1\r
2,Aviation & Transport,CAT2\r
3,Banking & Finance,CAT3\r
4,Cafes & Restaurants & Bars,CAT4\r


1.THE OLDEST BUISNESS IN THE WORLD

In [6]:
query1 = text("""SELECT MIN(year_founded) AS oldest_founding_year,
             MAX(year_founded) AS newest_founding_year
             FROM business;""")

In [7]:
result = connection.execute(query1)
df1 = pd.DataFrame(result.fetchall())
df1

,oldest_founding_year,newest_founding_year
0,578,1999


2.HOW MANY BUSINESSES WERE FOUNDED BEFORE 1000?

In [8]:
query2=text(""" SELECT COUNT(*) AS business_found_before_1000
            FROM business
            WHERE year_founded < 1000; """)

In [9]:
result = connection.execute(query2)
df2 = pd.DataFrame(result.fetchall())
df2

,business_found_before_1000
0,6


3.WHICH BUSINESSES WERE FOUND BEFORE 1000?

In [10]:
query3=text(""" SELECT *
            FROM business
            WHERE year_founded < 1000
            ORDER  BY year_founded ASC; """)

In [11]:
result = connection.execute(query3)
df3 = pd.DataFrame(result.fetchall())
df3

,business,year_founded,country_code,category_code
0,Kongō Gumi,578,JPN,CAT6\r
1,St. Peter Stifts Kulinarium,803,AUT,CAT4\r
2,Staffelter Hof Winery,862,DEU,CAT9\r
3,Monnaie de Paris,864,FRA,CAT12\r
4,The Royal Mint,886,GBR,CAT12\r
5,Sean's Bar,900,IRL,CAT4\r


4.EXPLORING THE CATEGORIES

In [12]:
query4 = text("""SELECT b.business, b.year_founded, b.country_code, c.category
              FROM business AS b
              JOIN categories AS c 
              ON c.category_code = b.category_code
              WHERE b.year_founded < 1000
              ORDER BY year_founded ASC;""")

In [13]:
result = connection.execute(query4)
df4 = pd.DataFrame(result.fetchall())
df4

,business,year_founded,country_code,category
0,Kongō Gumi,578,JPN,Construction
1,St. Peter Stifts Kulinarium,803,AUT,Cafes & Restaurants & Bars
2,Staffelter Hof Winery,862,DEU,Distillers & Vintners & Breweries
3,Monnaie de Paris,864,FRA,Manufacturing & Production
4,The Royal Mint,886,GBR,Manufacturing & Production
5,Sean's Bar,900,IRL,Cafes & Restaurants & Bars


5.COUNTING THE CATEGORIES

In [14]:
query5 = text(""" WITH filtered_business AS (
                    SELECT b.business, b.year_founded, b.country_code, c.category
                    FROM business AS b
                    JOIN categories AS c ON b.category_code = c.category_code)
              SELECT category , COUNT(*) AS n
              FROM filtered_business
              GROUP BY category
              ORDER BY n DESC
              LIMIT 10;""")

In [15]:
result = connection.execute(query5)
df5 = pd.DataFrame(result.fetchall())
df5

,category,n
0,Banking & Finance,37
1,Distillers & Vintners & Breweries,22
2,Aviation & Transport,19
3,Postal Service,16
4,Manufacturing & Production,15
5,Media,7
6,Food & Beverages,6
7,Agriculture,6
8,Cafes & Restaurants & Bars,6
9,Energy,4


6.OLDEST BUSSINESS BY CONTINENT

In [16]:
query6 = text(""" SELECT MIN(b.year_founded) AS oldest, c.continent
              FROM business as b
              JOIN countries as c 
              ON b.country_code = c.country_code
              GROUP BY c.continent
              ORDER BY oldest ASC;""")

In [17]:
result = connection.execute(query6)
df6 = pd.DataFrame(result.fetchall())
df6

,oldest,continent
0,578,Asia
1,803,Europe
2,1534,North America
3,1565,South America
4,1772,Africa
5,1809,Oceania


7.JOINING EVERYTHING FOR FURTHER ANALYSIS

In [18]:
query7 = text(""" SELECT b.business , b.year_founded, cat.category, cont.country, cont.continent
              FROM business as b
              JOIN categories as cat
              ON b.category_code = cat.category_code
              JOIN countries as cont
              ON b.country_code = cont.country_code; """)

In [19]:
result = connection.execute(query7)
df7 = pd.DataFrame(result.fetchall())
df7

,business,year_founded,category,country,continent
0,Spinzar Cotton Company,1930,Agriculture,Afghanistan,Asia
1,ALBtelecom,1912,Telecommunications,Albania,Europe
2,Andbank,1930,Banking & Finance,Andorra,Europe
3,Liwa Chemicals,1939,Manufacturing & Production,United Arab Emirates,Asia
4,Bank of the Province of Buenos Aires,1822,Banking & Finance,Argentina,South America
...,...,...,...,...,...
158,Yemenia Airways,1962,Aviation & Transport,Yemen,Asia
159,Premier FMCG,1820,Manufacturing & Production,South Africa,Africa
160,ZamPost,1896,Postal Service,Zambia,Africa
161,Standard Chartered Zimbabwe,1892,Banking & Finance,Zimbabwe,Africa


8.COUNTING CATEGORIES BY CONTINENT

In [20]:
query8 = text(""" WITH merged_db AS (
                      SELECT b.business , b.year_founded, cat.category, cont.country, cont.continent
                      FROM business as b
                      JOIN categories as cat
                      ON b.category_code = cat.category_code
                      JOIN countries as cont
                      ON b.country_code = cont.country_code )
              SELECT COUNT(business) AS number_of_businesses , continent, category
              FROM merged_db
              GROUP by continent, category; """)

In [21]:
result = connection.execute(query8)
df8 = pd.DataFrame(result.fetchall())
df8.head()

,number_of_businesses,continent,category
0,1,Asia,Agriculture
1,1,Europe,Telecommunications
2,5,Europe,Banking & Finance
3,3,Asia,Manufacturing & Production
4,3,South America,Banking & Finance


9.FILTERING COUNTS BY CONTINENT AND CATEGORY

In [22]:
query9 = text(""" WITH merged_db AS (
                      SELECT b.business , b.year_founded, cat.category, cont.country, cont.continent
                      FROM business as b
                      JOIN categories as cat
                      ON b.category_code = cat.category_code
                      JOIN countries as cont
                      ON b.country_code = cont.country_code )
              SELECT COUNT(business) AS number_of_businesses , continent, category
              FROM merged_db
              GROUP by continent, category
              HAVING number_of_businesses > 5; """)

In [23]:
result = connection.execute(query9)
df9 = pd.DataFrame(result.fetchall())
df9.head()

,number_of_businesses,continent,category
0,7,Asia,Aviation & Transport
1,12,Europe,Distillers & Vintners & Breweries
2,10,Africa,Aviation & Transport
3,17,Africa,Banking & Finance
4,9,Africa,Postal Service
